# Introduction 

In this Jupyter notebook, I present my approach to the recent Kaggle competition, [Facebook Recruiting IV: Human or Robot?](https://www.kaggle.com/c/facebook-recruiting-iv-human-or-bot). 

# Problem

For this competition, we will be chasing down robots for an online auction site. Human bidders on the site are becoming increasingly frustrated with their inability to win auctions vs their software-controlled counterparts. Goal of this competition is to identify online auction bids that are placed by bots, helping the site owners to easily flag these users for removal from their site to prevent unfair auction activity.

# Libaries 

For the most part, I used Python standard libraries and the scientific Python libraries available in the Anaconda distribution (pandas, scikit-learn, scipy and numpy). 
For the libaries, I used the standard libaries in Python ( pandas , numpy , scikit-learn ).


In [3]:
import numpy as np
import pandas as  pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from catboost import CatBoostClassifier      
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
import statistics
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import roc_auc_score

# Feature Engineering

## Trdr;
Some of the features we came up with was : <br>
Num of bids<br>
statistics of time difference for bids of each bidder<br>
entropy level of each bidder<br>
standard statistics of characteristics of each bidder (IPs, Devices, Countries, URLs)<br>
Number of 'first' and 'last' bids done for an auction for each bidder


In [4]:
import pandas as  pd
bids_df= pd.read_csv('data/bids.csv')
df_train= pd.read_csv('data/train.csv')
test_df= pd.read_csv('data/test.csv')


In [5]:
bids_df.head()

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,9759243157894736,py,112.54.208.157,vasstdc27m7nks3
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,9759243157894736,in,18.99.175.133,vasstdc27m7nks3
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,9759243157894736,in,145.138.5.37,vasstdc27m7nks3


# Converting columns to numeric values & Computing Bids Per Auction

In [6]:
bids_unique= bids_df.groupby('bidder_id').nunique()

In [7]:
bids_unique = bids_unique.drop(["bid_id",'bidder_id'],axis = 1)

In [8]:
join_result= df_train.merge(bids_unique, how= 'left', on = 'bidder_id')
join_result.fillna(0)

,bidder_id,payment_account,address,outcome,auction,merchandise,device,time,country,ip,url
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,18.0,1.0,14.0,24.0,6.0,20.0,1.0
1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,1.0,1.0,2.0,3.0,1.0,3.0,2.0
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,1.0,2.0,4.0,1.0,4.0,2.0
3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,23.0,1.0,53.0,155.0,2.0,123.0,91.0
5,7eaefc97fbf6af12e930528151f86eb91bafh,a3d2de7675556553a5f08e4c88d2c228yory1,5a1d8f28bc31aa6d72bef2d8fbf48b967hra3,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,25558d24bca82beef0f9db4ba1fe2045ynnvq,81580585d4dedd473da11aabf37fe9d4e2s2n,9a6d81115b9b653ba326eb510e9163b47drqj,0.0,8.0,1.0,2.0,8.0,1.0,3.0,8.0
7,88ae7a35e374a6fddd079ebb28c822eeohwse,a3d2de7675556553a5f08e4c88d2c2289zref,3a7e6a32b24aeab0688e91a41f3188e22iuec,0.0,10.0,1.0,3.0,34.0,3.0,32.0,1.0
8,57db69e32163f3e486dc6ef7d615aa12usje6,bf1c3151cc309308077ad0ccb99779ad12apw,31b95425d178b89fd7306762bb48bfb5n04sj,0.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0
9,d1be739798ba0745a1fd72ac918a9f1929hei,f49162ea9903fc00e4721d2f7972df9d6az4s,5b1f6e97a1cc27cd7fa9a3fe17eccd2a6mpdv,0.0,10.0,1.0,11.0,14.0,5.0,12.0,7.0


In [9]:
bids_df_count= bids_df.groupby("bidder_id")
counts= bids_df_count['url'].count().reset_index().rename(columns = {'url':'num_bids'})

In [10]:
new_train= join_result.merge(counts, how= 'left')

In [11]:
new_train.head()

,bidder_id,payment_account,address,outcome,auction,merchandise,device,time,country,ip,url,num_bids
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,18.0,1.0,14.0,24.0,6.0,20.0,1.0,24.0
1,624f258b49e77713fc34034560f93fb3hu3jo,a3d2de7675556553a5f08e4c88d2c228v1sga,ae87054e5a97a8f840a3991d12611fdcrfbq3,0.0,1.0,1.0,2.0,3.0,1.0,3.0,2.0,3.0
2,1c5f4fc669099bfbfac515cd26997bd12ruaj,a3d2de7675556553a5f08e4c88d2c2280cybl,92520288b50f03907041887884ba49c0cl0pd,0.0,4.0,1.0,2.0,4.0,1.0,4.0,2.0,4.0
3,4bee9aba2abda51bf43d639013d6efe12iycd,51d80e233f7b6a7dfdee484a3c120f3b2ita8,4cb9717c8ad7e88a9a284989dd79b98dbevyi,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,4ab12bc61c82ddd9c2d65e60555808acqgos1,a3d2de7675556553a5f08e4c88d2c22857ddh,2a96c3ce94b3be921e0296097b88b56a7x1ji,0.0,23.0,1.0,53.0,155.0,2.0,123.0,91.0,155.0


# Calculating time difference for bidders' bids 

In [12]:
bids_df_sorted= bids_df.sort_values(by= ['bidder_id', 'time'], ascending = [True, True])

In [13]:
timediff= bids_df_sorted.groupby('bidder_id')['time'].diff()

In [14]:
bids_df_sorted['timediff']= timediff

In [15]:
bids= pd.DataFrame(data = bids_df_sorted['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df_sorted['bidder_id'].unique())

In [16]:
max_time = bids_df_sorted.groupby('bidder_id')['time'].max()
bids['maxtime_num'] = max_time
min_time = bids_df_sorted.groupby('bidder_id')['time'].min()
bids['mintime_num'] = min_time

In [17]:
max_diff = bids_df_sorted.groupby('bidder_id')['timediff'].max()
max_diff = max_diff.fillna(max_diff.mean())
bids['maxdiff_num'] = max_diff
min_diff = bids_df_sorted.groupby('bidder_id')['timediff'].max()
min_diff = min_diff.fillna(min_diff.mean())
bids['mindiff_num'] = min_diff
mean_diff = bids_df_sorted.groupby('bidder_id')['timediff'].mean()
mean_diff = mean_diff.fillna(mean_diff.mean())
bids['meandiff_num'] = mean_diff
median_diff = bids_df_sorted.groupby('bidder_id')['timediff'].median()
median_diff = median_diff.fillna(median_diff.mean())
bids['mediandiff_num'] = median_diff
std_diff = bids_df_sorted.groupby('bidder_id')['timediff'].std()
std_diff = std_diff.fillna(std_diff.mean())
bids['std_diff'] = std_diff

In [18]:
end_auction_time= bids_df.groupby('auction').time.max().reset_index()
end_auction_time= end_auction_time.rename(columns= {'time': 'endtime'})
start_auction_time= bids_df.groupby('auction').time.min().reset_index()
start_auction_time= start_auction_time.rename(columns= {'time': 'starttime'})
start_end_times= pd.merge(start_auction_time, end_auction_time, on= 'auction', how= 'left')

In [19]:
bids_df_sorted= pd.merge(bids_df_sorted, start_end_times[['auction', 'starttime','endtime']], on='auction', how= 'left')

In [20]:
bids_df_sorted['time_until_end']= bids_df_sorted.endtime- bids_df_sorted.time
bids_df_sorted['time_from_start']= bids_df_sorted.time - bids_df_sorted.starttime

In [21]:
median_start= bids_df_sorted.groupby('bidder_id').time_from_start.median()
bids['median_start'] = median_start
median_start= median_start.fillna(median_start.mean())
mean_start= bids_df_sorted.groupby('bidder_id').time_from_start.mean()
bids['mean_start'] = mean_start
mean_start= mean_start.fillna(mean_start.mean())
std_start= bids_df_sorted.groupby('bidder_id').time_from_start.std()
std_start= std_start.fillna(std_start.mean())
bids['std_start'] = std_start

In [22]:
median_end= bids_df_sorted.groupby('bidder_id').time_until_end.median()
bids['median_end'] = median_end
median_end= median_end.fillna(median_end.mean())
mean_end= bids_df_sorted.groupby('bidder_id').time_until_end.mean()
bids['mean_end'] = mean_end
mean_end= mean_end.fillna(mean_end.mean())
std_end= bids_df_sorted.groupby('bidder_id').time_until_end.std()
std_end= std_end.fillna(std_end.mean())
bids['std_end'] = std_end

# Calculating Entropy for each bidder

In [23]:
def log_entropy(x):
    e = np.sum(np.log(np.array(range(1,np.sum(x)))))
    for i in x:
        e -= np.sum(np.log(np.array(range(1,i))))

    return e


In [24]:
dataweusing= bids_df_sorted[['bidder_id', 'auction', 'ip', 'url', 'country', 'device', 'merchandise']]
dataweusing= dataweusing.groupby(['bidder_id', 'auction'])

In [25]:
dataweusing.head()

,bidder_id,auction,ip,url,country,device,merchandise
0,001068c415025a009fee375a12cff4fcnht8y,4ifac,139.226.147.115,vasstdc27m7nks3,bn,phone561,jewelry
1,002d229ffb247009810828f648afc2ef593rb,2tdw2,37.40.254.131,vasstdc27m7nks3,sg,phone640,mobile
2,002d229ffb247009810828f648afc2ef593rb,2tdw2,37.40.254.131,vasstdc27m7nks3,sg,phone219,mobile
3,0030a2dd87ad2733e0873062e4f83954mkj86,obbny,21.67.17.162,vnw40k8zzokijsv,ir,phone313,mobile
4,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,obbny,44.241.8.179,sj4jidex850loas,id,phone420,mobile
5,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,cqsh6,190.88.89.83,sj4jidex850loas,id,phone102,mobile
6,003180b29c6a5f8f1d84a6b7b6f7be57tjj1o,efh5o,115.47.140.180,vasstdc27m7nks3,id,phone257,mobile
7,00486a11dff552c4bd7696265724ff81yeo9v,no958,143.118.40.162,vasstdc27m7nks3,ng,phone4,home goods
8,00486a11dff552c4bd7696265724ff81yeo9v,6plix,143.118.40.162,vasstdc27m7nks3,ng,phone4,home goods
9,00486a11dff552c4bd7696265724ff81yeo9v,gst86,54.212.177.220,0wfuwlacucr1cdl,ng,phone45,home goods


In [26]:
bids_per_auction_per_ip= bids_df_sorted[['bidder_id', 'auction', 'ip']].groupby(['bidder_id','auction', 'ip']).size()
bids_per_auction_per_ip2= bids_per_auction_per_ip.groupby(['bidder_id', 'auction']).apply(log_entropy).reset_index()
median_entropy=bids_per_auction_per_ip2.groupby('bidder_id').median()
mean_entropy=bids_per_auction_per_ip2.groupby('bidder_id').mean()
std_entropy=bids_per_auction_per_ip2.groupby('bidder_id').std()
bids['mean_entropy']= mean_entropy
bids['median_entropy']= median_entropy
bids['std_entropy']= std_entropy

# Calculating IPs, countries,devices,urls per bidder per auction

In [27]:
ips_per_bidder_per_auction= dataweusing.ip.nunique()

In [28]:
median_ips_per_bidder_per_auction= ips_per_bidder_per_auction.groupby('bidder_id').median()
mean_ips_per_bidder_per_auction=ips_per_bidder_per_auction.groupby('bidder_id').mean()
std_ips_per_bidder_per_auction=ips_per_bidder_per_auction.groupby('bidder_id').std()
bids['median_ips_per_bidder_per_auction']= median_ips_per_bidder_per_auction
bids['mean_ips_per_bidder_per_auction']= mean_ips_per_bidder_per_auction
bids['std_ips_per_bidder_per_auction']= std_ips_per_bidder_per_auction

ip_count= bids_df_sorted.groupby(['bidder_id', 'ip']).size()
ip_entropy= ip_count.groupby('bidder_id').apply(log_entropy)
bids['ip_entropy']= ip_entropy

In [29]:
country_per_bidder_per_auction= dataweusing.country.nunique()

In [30]:
median_country_per_bidder_per_auction= country_per_bidder_per_auction.groupby('bidder_id').median()
mean_country_per_bidder_per_auction=country_per_bidder_per_auction.groupby('bidder_id').mean()
std_country_per_bidder_per_auction=country_per_bidder_per_auction.groupby('bidder_id').std()
bids['median_country_per_bidder_per_auction']= median_country_per_bidder_per_auction
bids['mean_country_per_bidder_per_auction']= mean_country_per_bidder_per_auction
bids['std_country_per_bidder_per_auction']= std_country_per_bidder_per_auction
country_count= bids_df_sorted.groupby(['bidder_id', 'country']).size()
country_entropy= ip_count.groupby('bidder_id').apply(log_entropy)
bids['country_entropy']= country_entropy

In [31]:
device_per_bidder_per_auction= dataweusing.device.nunique()

In [32]:
median_devices_per_bidder_per_auction= device_per_bidder_per_auction.groupby('bidder_id').median()
mean_devices_per_bidder_per_auction=device_per_bidder_per_auction.groupby('bidder_id').mean()
std_devices_per_bidder_per_auction=device_per_bidder_per_auction.groupby('bidder_id').std()
bids['median_devices_per_bidder_per_auction']= median_devices_per_bidder_per_auction
bids['mean_devices_per_bidder_per_auction']= mean_devices_per_bidder_per_auction
bids['std_devices_per_bidder_per_auction']= std_devices_per_bidder_per_auction
device_count= bids_df_sorted.groupby(['bidder_id', 'device']).size()
device_entropy= ip_count.groupby('bidder_id').apply(log_entropy)
bids['device_entropy']= device_entropy

In [33]:
url_per_bidder_per_auction= dataweusing.url.nunique()

In [34]:
median_url_per_bidder_per_auction= url_per_bidder_per_auction.groupby('bidder_id').median()
mean_url_per_bidder_per_auction=url_per_bidder_per_auction.groupby('bidder_id').mean()
std_url_per_bidder_per_auction=url_per_bidder_per_auction.groupby('bidder_id').std()
bids['median_url_per_bidder_per_auction']= median_url_per_bidder_per_auction
bids['mean_url_per_bidder_per_auction']= mean_url_per_bidder_per_auction
bids['std_url_per_bidder_per_auction']= std_url_per_bidder_per_auction
url_count= bids_df_sorted.groupby(['bidder_id', 'url']).size()
url_entropy= ip_count.groupby('bidder_id').apply(log_entropy)
bids['url_entropy']= device_entropy

In [35]:
new_train= new_train.merge(bids, how = 'left', left_on = ['bidder_id'], right_on = ['bidder_id'])
new_train.corr()
new_train['country'].replace({0:1},inplace=True)

In [36]:
new_train['bids_per_auction'] = new_train['num_bids']/new_train['auction']
new_train['bids_per_url'] = new_train['num_bids']/new_train['url']
new_train['bids_per_ip'] = new_train['num_bids']/new_train['ip']
new_train['bids_per_country'] = new_train['num_bids']/new_train['country']
new_train['bids_per_device'] = new_train['num_bids']/new_train['device']


In [37]:
new_train.columns

Index(['bidder_id', 'payment_account', 'address', 'outcome', 'auction',
       'merchandise', 'device', 'time', 'country', 'ip', 'url', 'num_bids',
       'maxtime_num', 'mintime_num', 'maxdiff_num', 'mindiff_num',
       'meandiff_num', 'mediandiff_num', 'std_diff', 'mean_entropy',
       'median_entropy', 'std_entropy', 'median_ips_per_bidder_per_auction',
       'mean_ips_per_bidder_per_auction', 'std_ips_per_bidder_per_auction',
       'ip_entropy', 'median_country_per_bidder_per_auction',
       'mean_country_per_bidder_per_auction',
       'std_country_per_bidder_per_auction', 'country_entropy',
       'median_devices_per_bidder_per_auction',
       'mean_devices_per_bidder_per_auction',
       'std_devices_per_bidder_per_auction', 'device_entropy',
       'median_url_per_bidder_per_auction', 'mean_url_per_bidder_per_auction',
       'std_url_per_bidder_per_auction', 'url_entropy', 'bids_per_auction',
       'bids_per_url', 'bids_per_ip', 'bids_per_country', 'bids_per_device'],
 

# Calculating country, devices, ips , urls per auction per bidder

In [38]:
diversity_bids = bids_df.sort_values(['bidder_id','country'])

In [39]:
num_countries_per_auction = diversity_bids.groupby(['bidder_id','auction'])['country'].nunique()
num_countries_per_auction = pd.DataFrame(num_countries_per_auction)

In [40]:
mean_country_per_auction = num_countries_per_auction.groupby("bidder_id").mean()
mean_country_per_auction = mean_country_per_auction.reset_index().rename(columns = {'country':"mean_country_per_auction"})

max_country_per_auction = num_countries_per_auction.groupby("bidder_id").max()
max_country_per_auction = max_country_per_auction.reset_index().rename(columns = {'country':"max_country_per_auction"})

min_country_per_auction = num_countries_per_auction.groupby("bidder_id").min()
min_country_per_auction = min_country_per_auction.reset_index().rename(columns = {'country':"min_country_per_auction"})

std_country_per_auction = num_countries_per_auction.groupby("bidder_id").std()
std_country_per_auction = std_country_per_auction.reset_index().rename(columns = {'country':"std_country_per_auction"})


In [41]:
country_bids = pd.DataFrame(data = bids_df_sorted['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df_sorted['bidder_id'].unique())

In [42]:
country_bids = country_bids.merge(mean_country_per_auction,on='bidder_id',how='left')
country_bids = country_bids.merge(max_country_per_auction,on='bidder_id',how='left')
country_bids = country_bids.merge(min_country_per_auction,on='bidder_id',how='left')
country_bids = country_bids.merge(std_country_per_auction,on='bidder_id',how='left')

In [43]:
device_bids = bids_df.sort_values(['bidder_id','device'])
num_devices_per_auction = device_bids.groupby(['bidder_id','auction'])['device'].nunique()
num_devices_per_auction = pd.DataFrame(num_devices_per_auction)

In [44]:
mean_devices_per_auction = num_devices_per_auction.groupby("bidder_id").mean()
mean_devices_per_auction = mean_devices_per_auction.reset_index().rename(columns = {'device':"mean_devices_per_auction"})

max_devices_per_auction = num_devices_per_auction.groupby("bidder_id").max()
max_devices_per_auction = max_devices_per_auction.reset_index().rename(columns = {'device':"max_devices_per_auction"})

min_devices_per_auction = num_devices_per_auction.groupby("bidder_id").min()
min_devices_per_auction = min_devices_per_auction.reset_index().rename(columns = {'device':"min_devices_per_auction"})

std_devices_per_auction = num_devices_per_auction.groupby("bidder_id").std()
std_devices_per_auction = std_devices_per_auction.reset_index().rename(columns = {'device':"std_devices_per_auction"})

In [45]:
device_bids = pd.DataFrame(data = bids_df_sorted['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df_sorted['bidder_id'].unique())

In [46]:
device_bids = device_bids.merge(mean_devices_per_auction,on='bidder_id',how='left')
device_bids = device_bids.merge(max_devices_per_auction,on='bidder_id',how='left')
device_bids = device_bids.merge(min_devices_per_auction,on='bidder_id',how='left')
device_bids = device_bids.merge(std_devices_per_auction,on='bidder_id',how='left')

In [47]:
device_bids.fillna(0,inplace=True)

In [48]:
ip_bids = bids_df.sort_values(['bidder_id','ip'])
num_ip_per_auction = ip_bids.groupby(['bidder_id','auction'])['ip'].nunique()
num_ip_per_auction = pd.DataFrame(num_ip_per_auction)

In [49]:
mean_ip_per_auction = num_ip_per_auction.groupby("bidder_id").mean()
mean_ip_per_auction = mean_ip_per_auction.reset_index().rename(columns = {'ip':"mean_ip_per_auction"})

max_ip_per_auction = num_ip_per_auction.groupby("bidder_id").max()
max_ip_per_auction = max_ip_per_auction.reset_index().rename(columns = {'ip':"max_ip_per_auction"})

min_ip_per_auction = num_ip_per_auction.groupby("bidder_id").min()
min_ip_per_auction = min_ip_per_auction.reset_index().rename(columns = {'ip':"min_ip_per_auction"})

std_ip_per_auction = num_ip_per_auction.groupby("bidder_id").std()
std_ip_per_auction = std_ip_per_auction.reset_index().rename(columns = {'ip':"std_ip_per_auction"})

In [50]:
ip_bids = pd.DataFrame(data = bids_df_sorted['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df_sorted['bidder_id'].unique())

In [51]:
ip_bids = ip_bids.merge(mean_ip_per_auction,on='bidder_id',how='left')
ip_bids = ip_bids.merge(max_ip_per_auction,on='bidder_id',how='left')
ip_bids = ip_bids.merge(min_ip_per_auction,on='bidder_id',how='left')
ip_bids = ip_bids.merge(std_ip_per_auction,on='bidder_id',how='left')

In [52]:
ip_bids.fillna(0,inplace=True)

In [53]:
url_bids= bids_df.sort_values(['bidder_id','url'])
num_url_per_auction = url_bids.groupby(['bidder_id','auction'])['url'].nunique()
num_url_per_auction = pd.DataFrame(num_url_per_auction)

In [54]:
mean_url_per_auction = num_url_per_auction.groupby("bidder_id").mean()
mean_url_per_auction = mean_url_per_auction.reset_index().rename(columns = {'url':"mean_url_per_auction"})

max_url_per_auction = num_url_per_auction.groupby("bidder_id").max()
max_url_per_auction = max_url_per_auction.reset_index().rename(columns = {'url':"max_url_per_auction"})

min_url_per_auction = num_url_per_auction.groupby("bidder_id").min()
min_url_per_auction = min_url_per_auction.reset_index().rename(columns = {'url':"min_url_per_auction"})

std_url_per_auction = num_url_per_auction.groupby("bidder_id").std()
std_url_per_auction = std_url_per_auction.reset_index().rename(columns = {'url':"std_url_per_auction"})


In [55]:
url_bids = pd.DataFrame(data = bids_df_sorted['bidder_id'].unique(), columns = ['bidder_id'],
                    index = bids_df_sorted['bidder_id'].unique())

In [56]:
url_bids = url_bids.merge(mean_url_per_auction,on='bidder_id',how='left')
url_bids = url_bids.merge(max_url_per_auction,on='bidder_id',how='left')
url_bids = url_bids.merge(min_url_per_auction,on='bidder_id',how='left')
url_bids = url_bids.merge(std_url_per_auction,on='bidder_id',how='left')

In [57]:
url_bids.fillna(0,inplace=True)

# Making of training dataset

In [58]:
train_model= new_train.copy()

In [59]:
train_model= train_model.merge(country_bids,on='bidder_id',how='left')

In [60]:
train_model= train_model.merge(device_bids,on='bidder_id',how='left')

In [61]:
train_model= train_model.merge(ip_bids,on='bidder_id',how='left')

In [62]:
train_model= train_model.merge(url_bids,on='bidder_id',how='left')

In [63]:
try_auction = bids_df.groupby(['auction'])

# Checking of how many 'first's and 'last's bids each bidder made

In [64]:
first_bid_list = set(try_auction.first()['bidder_id'])
last_bid_list = set(try_auction.last()['bidder_id'])

In [65]:
def check_first_bid(dummy):
    count = 0
    for bidder_id in first_bid_list:
        if(dummy == bidder_id):
            count+=1
    return count
        
def check_last_bid(dummy):
    count = 0
    for bidder_id in last_bid_list:
        if(dummy == bidder_id):
            count+=1
    return count
        
    

In [66]:
train_model['check_first'] = train_model['bidder_id'].apply(func = check_first_bid)
train_model['check_last'] = train_model['bidder_id'].apply(func = check_last_bid)

In [67]:
train_model.fillna(0,inplace=True)

In [68]:
bots_train_model = train_model[train_model['outcome'] == 1]
humans_train_model = train_model[train_model['outcome'] == 0]
bots_train_model = bots_train_model[bots_train_model['num_bids'] > 1]

In [69]:
train_model = pd.concat([bots_train_model,humans_train_model])

In [70]:
train_model = train_model.reset_index()

In [71]:
train_model.drop(columns = ['index'],inplace = True)

In [72]:
corr = train_model.corr()

corr[:1].T

,outcome
outcome,1.000000
auction,0.139967
merchandise,0.026011
device,0.114313
time,0.060833
country,0.140476
ip,0.087664
url,0.019616
num_bids,0.041530
maxtime_num,0.028434


# Training of model with training data set and testing out features

In [73]:
y = train_model['outcome']
X = train_model.drop(columns= [
                                     'outcome', 
                                     'bidder_id',
                                     'payment_account',
                                     'address', 
                                     
                                    
                                     ])

In [74]:
model1 = RandomForestClassifier(n_estimators=500, max_features=35,
                                   max_depth=8, random_state=20,
                                   criterion='entropy',)

model2 = xgb.XGBClassifier( objective= 'binary:logistic', nthread= 10,
                              eval_metric= 'auc', silent= 1, seed= 20,

                              max_depth= 6, gamma= 0, base_score= 0.50,
                              min_child_weight= 4, subsample= 0.5,
                              colsample_bytree= 1, eta= 0.01,)

model3 = GradientBoostingClassifier(n_estimators=200,
                                       random_state=20,
                                       max_depth=5,
                                       learning_rate=0.03,
                                       max_features=5,)
model4 = CatBoostClassifier(random_state=20)

k_fold_accuracy = []
kf = KFold(n_splits=10,shuffle = True)

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model1.fit(X_train,y_train)
    model2.fit(X_train,y_train)
    model3.fit(X_train,y_train)
    model4.fit(X_train,y_train)


    y_pred1 = model1.predict_proba(X_test)
    y_pred2 = model2.predict_proba(X_test)
    y_pred3 = model3.predict_proba(X_test)
    y_pred4 = model4.predict_proba(X_test)

    
    accu1 = model1.predict(X_test)
    accu2 = model2.predict(X_test)
    accu3 = model3.predict(X_test)
    accu4 = model4.predict(X_test)

C:\Users\user\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:08:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Learning rate set to 0.013263
0:	learn: 0.6720898	total: 183ms	remaining: 3m 2s
1:	learn: 0.6439968	total: 194ms	remaining: 1m 36s
2:	learn: 0.6185590	total: 204ms	remaining: 1m 7s
3:	learn: 0.5921036	total: 210ms	remaining: 52.4s
4:	learn: 0.5735164	total: 217ms	remaining: 43.1s
5:	learn: 0.5494438	total: 223ms	remaining: 37s
6:	learn: 0.5319994	total: 229ms	remaining: 32.5s
7:	learn: 0.5112995	total: 236ms	remaining: 29.2s
8:	learn: 0.4997922	total: 237ms	remaining: 26.1s
9:	learn: 0.4826432	total: 244ms	remaining: 24.1s
10:	learn: 0.4653085	total: 250ms	remaining: 22.5s
11:	learn: 0.4498872	total: 256ms	re

169:	learn: 0.0805146	total: 1.27s	remaining: 6.21s
170:	learn: 0.0800887	total: 1.28s	remaining: 6.21s
171:	learn: 0.0797441	total: 1.29s	remaining: 6.2s
172:	learn: 0.0795707	total: 1.3s	remaining: 6.2s
173:	learn: 0.0793474	total: 1.3s	remaining: 6.18s
174:	learn: 0.0790104	total: 1.31s	remaining: 6.17s
175:	learn: 0.0788534	total: 1.32s	remaining: 6.16s
176:	learn: 0.0786020	total: 1.32s	remaining: 6.15s
177:	learn: 0.0783331	total: 1.33s	remaining: 6.14s
178:	learn: 0.0780131	total: 1.33s	remaining: 6.12s
179:	learn: 0.0777786	total: 1.34s	remaining: 6.11s
180:	learn: 0.0775850	total: 1.35s	remaining: 6.1s
181:	learn: 0.0773413	total: 1.35s	remaining: 6.08s
182:	learn: 0.0771479	total: 1.36s	remaining: 6.07s
183:	learn: 0.0769037	total: 1.37s	remaining: 6.06s
184:	learn: 0.0768365	total: 1.37s	remaining: 6.05s
185:	learn: 0.0765405	total: 1.38s	remaining: 6.04s
186:	learn: 0.0763247	total: 1.39s	remaining: 6.02s
187:	learn: 0.0760911	total: 1.39s	remaining: 6.01s
188:	learn: 0.075

344:	learn: 0.0503362	total: 2.51s	remaining: 4.77s
345:	learn: 0.0502777	total: 2.52s	remaining: 4.76s
346:	learn: 0.0501682	total: 2.53s	remaining: 4.76s
347:	learn: 0.0500187	total: 2.54s	remaining: 4.75s
348:	learn: 0.0499325	total: 2.55s	remaining: 4.75s
349:	learn: 0.0498401	total: 2.55s	remaining: 4.74s
350:	learn: 0.0497322	total: 2.56s	remaining: 4.74s
351:	learn: 0.0495973	total: 2.57s	remaining: 4.73s
352:	learn: 0.0494546	total: 2.58s	remaining: 4.73s
353:	learn: 0.0493850	total: 2.59s	remaining: 4.72s
354:	learn: 0.0493478	total: 2.6s	remaining: 4.71s
355:	learn: 0.0492274	total: 2.6s	remaining: 4.71s
356:	learn: 0.0491341	total: 2.61s	remaining: 4.7s
357:	learn: 0.0490121	total: 2.62s	remaining: 4.69s
358:	learn: 0.0489080	total: 2.63s	remaining: 4.69s
359:	learn: 0.0488299	total: 2.63s	remaining: 4.68s
360:	learn: 0.0487516	total: 2.64s	remaining: 4.67s
361:	learn: 0.0486744	total: 2.65s	remaining: 4.67s
362:	learn: 0.0486108	total: 2.66s	remaining: 4.66s
363:	learn: 0.0

524:	learn: 0.0349348	total: 3.75s	remaining: 3.39s
525:	learn: 0.0348592	total: 3.76s	remaining: 3.39s
526:	learn: 0.0348154	total: 3.76s	remaining: 3.38s
527:	learn: 0.0347320	total: 3.77s	remaining: 3.37s
528:	learn: 0.0346330	total: 3.78s	remaining: 3.36s
529:	learn: 0.0345609	total: 3.79s	remaining: 3.36s
530:	learn: 0.0344987	total: 3.79s	remaining: 3.35s
531:	learn: 0.0343945	total: 3.8s	remaining: 3.34s
532:	learn: 0.0342950	total: 3.81s	remaining: 3.33s
533:	learn: 0.0342112	total: 3.81s	remaining: 3.33s
534:	learn: 0.0341580	total: 3.82s	remaining: 3.32s
535:	learn: 0.0341014	total: 3.83s	remaining: 3.31s
536:	learn: 0.0340465	total: 3.83s	remaining: 3.31s
537:	learn: 0.0340032	total: 3.84s	remaining: 3.3s
538:	learn: 0.0339190	total: 3.85s	remaining: 3.29s
539:	learn: 0.0338061	total: 3.85s	remaining: 3.28s
540:	learn: 0.0337464	total: 3.86s	remaining: 3.28s
541:	learn: 0.0336897	total: 3.87s	remaining: 3.27s
542:	learn: 0.0336122	total: 3.88s	remaining: 3.26s
543:	learn: 0.

...

156:	learn: 0.0839985	total: 1.25s	remaining: 6.72s
157:	learn: 0.0837693	total: 1.26s	remaining: 6.72s
158:	learn: 0.0834458	total: 1.27s	remaining: 6.72s
159:	learn: 0.0831565	total: 1.28s	remaining: 6.72s
160:	learn: 0.0828480	total: 1.29s	remaining: 6.71s
161:	learn: 0.0825964	total: 1.3s	remaining: 6.71s
162:	learn: 0.0823571	total: 1.3s	remaining: 6.7s
163:	learn: 0.0819814	total: 1.31s	remaining: 6.7s
164:	learn: 0.0816425	total: 1.32s	remaining: 6.69s
165:	learn: 0.0814114	total: 1.33s	remaining: 6.68s
166:	learn: 0.0811131	total: 1.34s	remaining: 6.67s
167:	learn: 0.0807858	total: 1.34s	remaining: 6.66s
168:	learn: 0.0804131	total: 1.35s	remaining: 6.66s
169:	learn: 0.0802381	total: 1.36s	remaining: 6.65s
170:	learn: 0.0799509	total: 1.37s	remaining: 6.65s
171:	learn: 0.0796087	total: 1.38s	remaining: 6.64s
172:	learn: 0.0794224	total: 1.39s	remaining: 6.63s
173:	learn: 0.0792251	total: 1.4s	remaining: 6.64s
174:	learn: 0.0788797	total: 1.41s	remaining: 6.63s
175:	learn: 0.078

317:	learn: 0.0533379	total: 2.5s	remaining: 5.36s
318:	learn: 0.0532043	total: 2.5s	remaining: 5.35s
319:	learn: 0.0530188	total: 2.51s	remaining: 5.34s
320:	learn: 0.0528457	total: 2.52s	remaining: 5.33s
321:	learn: 0.0527732	total: 2.53s	remaining: 5.32s
322:	learn: 0.0525943	total: 2.53s	remaining: 5.31s
323:	learn: 0.0525626	total: 2.54s	remaining: 5.3s
324:	learn: 0.0523997	total: 2.55s	remaining: 5.29s
325:	learn: 0.0523560	total: 2.56s	remaining: 5.28s
326:	learn: 0.0521869	total: 2.56s	remaining: 5.27s
327:	learn: 0.0521465	total: 2.57s	remaining: 5.26s
328:	learn: 0.0520497	total: 2.58s	remaining: 5.26s
329:	learn: 0.0518976	total: 2.58s	remaining: 5.25s
330:	learn: 0.0516787	total: 2.59s	remaining: 5.24s
331:	learn: 0.0515507	total: 2.6s	remaining: 5.23s
332:	learn: 0.0514007	total: 2.61s	remaining: 5.22s
333:	learn: 0.0512524	total: 2.62s	remaining: 5.21s
334:	learn: 0.0510565	total: 2.62s	remaining: 5.21s
335:	learn: 0.0509299	total: 2.63s	remaining: 5.21s
336:	learn: 0.05

486:	learn: 0.0387962	total: 3.75s	remaining: 3.95s
487:	learn: 0.0387543	total: 3.76s	remaining: 3.94s
488:	learn: 0.0387076	total: 3.76s	remaining: 3.93s
489:	learn: 0.0386381	total: 3.77s	remaining: 3.92s
490:	learn: 0.0385974	total: 3.78s	remaining: 3.92s
491:	learn: 0.0385168	total: 3.78s	remaining: 3.91s
492:	learn: 0.0384625	total: 3.79s	remaining: 3.9s
493:	learn: 0.0384325	total: 3.8s	remaining: 3.89s
494:	learn: 0.0383432	total: 3.8s	remaining: 3.88s
495:	learn: 0.0382127	total: 3.81s	remaining: 3.87s
496:	learn: 0.0381720	total: 3.82s	remaining: 3.86s
497:	learn: 0.0381354	total: 3.83s	remaining: 3.85s
498:	learn: 0.0380584	total: 3.83s	remaining: 3.85s
499:	learn: 0.0379901	total: 3.84s	remaining: 3.84s
500:	learn: 0.0379735	total: 3.84s	remaining: 3.83s
501:	learn: 0.0379327	total: 3.85s	remaining: 3.82s
502:	learn: 0.0378589	total: 3.86s	remaining: 3.81s
503:	learn: 0.0377602	total: 3.86s	remaining: 3.8s
504:	learn: 0.0376643	total: 3.87s	remaining: 3.79s
505:	learn: 0.03

665:	learn: 0.0288161	total: 4.98s	remaining: 2.5s
666:	learn: 0.0287718	total: 4.99s	remaining: 2.49s
667:	learn: 0.0287023	total: 4.99s	remaining: 2.48s
668:	learn: 0.0286646	total: 5s	remaining: 2.47s
669:	learn: 0.0286419	total: 5.01s	remaining: 2.47s
670:	learn: 0.0286281	total: 5.01s	remaining: 2.46s
671:	learn: 0.0285580	total: 5.02s	remaining: 2.45s
672:	learn: 0.0285065	total: 5.03s	remaining: 2.44s
673:	learn: 0.0284786	total: 5.04s	remaining: 2.44s
674:	learn: 0.0284625	total: 5.04s	remaining: 2.43s
675:	learn: 0.0284248	total: 5.05s	remaining: 2.42s
676:	learn: 0.0283404	total: 5.05s	remaining: 2.41s
677:	learn: 0.0282887	total: 5.06s	remaining: 2.4s
678:	learn: 0.0282689	total: 5.07s	remaining: 2.4s
679:	learn: 0.0282517	total: 5.08s	remaining: 2.39s
680:	learn: 0.0281992	total: 5.08s	remaining: 2.38s
681:	learn: 0.0281646	total: 5.09s	remaining: 2.37s
682:	learn: 0.0281164	total: 5.09s	remaining: 2.36s
683:	learn: 0.0280591	total: 5.1s	remaining: 2.36s
684:	learn: 0.02798

845:	learn: 0.0220723	total: 6.22s	remaining: 1.13s
846:	learn: 0.0220433	total: 6.22s	remaining: 1.12s
847:	learn: 0.0219976	total: 6.23s	remaining: 1.12s
848:	learn: 0.0219664	total: 6.24s	remaining: 1.11s
849:	learn: 0.0219341	total: 6.25s	remaining: 1.1s
850:	learn: 0.0219235	total: 6.25s	remaining: 1.09s
851:	learn: 0.0218959	total: 6.26s	remaining: 1.09s
852:	learn: 0.0218679	total: 6.26s	remaining: 1.08s
853:	learn: 0.0218372	total: 6.27s	remaining: 1.07s
854:	learn: 0.0218062	total: 6.28s	remaining: 1.06s
855:	learn: 0.0217706	total: 6.29s	remaining: 1.06s
856:	learn: 0.0217184	total: 6.29s	remaining: 1.05s
857:	learn: 0.0217063	total: 6.3s	remaining: 1.04s
858:	learn: 0.0216455	total: 6.3s	remaining: 1.03s
859:	learn: 0.0216187	total: 6.31s	remaining: 1.03s
860:	learn: 0.0215914	total: 6.32s	remaining: 1.02s
861:	learn: 0.0215341	total: 6.32s	remaining: 1.01s
862:	learn: 0.0215240	total: 6.33s	remaining: 1s
863:	learn: 0.0214886	total: 6.34s	remaining: 998ms
864:	learn: 0.0214

# Analysing of Model Results

In [75]:
print('\nAccuracy_score on RandomForest : ', accuracy_score(y_test, accu1))
print('\nAccuracy_score on XGB : ', accuracy_score(y_test, accu2))
print('\nAccuracy_score on GradientBoost : ', accuracy_score(y_test, accu3))
print('\nAccuracy_score on CatBoost : ', accuracy_score(y_test, accu4))

averaging_accuracy = (accuracy_score(y_test, accu1) + accuracy_score(y_test, accu2) +accuracy_score(y_test, accu3)+accuracy_score(y_test, accu4))/4
print('\nAverage Accuracy : ', averaging_accuracy)



Accuracy_score on RandomForest :  0.97

Accuracy_score on XGB :  0.975

Accuracy_score on GradientBoost :  0.975

Accuracy_score on CatBoost :  0.975

Average Accuracy :  0.97375


In [76]:
print("AUC SCORES")
print("-----------------------------------")
print("RandomForest : " , roc_auc_score(y_test,y_pred1[:,1]))

print("XGB : " , roc_auc_score(y_test,y_pred2[:,1]))
print("Gradient Boost : " , roc_auc_score(y_test,y_pred3[:,1]))
print("Cat Boost : ", roc_auc_score(y_test,y_pred4[:,1]))

AUC SCORES
-----------------------------------
RandomForest :  0.9726128793486306
XGB :  0.9592894152479645
Gradient Boost :  0.9755736491487788
Cat Boost :  0.9792746113989638


In [77]:
score1 = roc_auc_score(y_test,y_pred1[:,1])
score2 = roc_auc_score(y_test,y_pred2[:,1])
score3 = roc_auc_score(y_test,y_pred3[:,1])
score4 = roc_auc_score(y_test,y_pred4[:,1])

In [78]:
print("\nAverage AUC :" , (score1+score2+score3+score4)/4)


Average AUC : 0.9716876387860844


# Executing predictions on final test dataset

In [79]:
final_test_df = test_df.merge(bids_unique,how= 'left',on = 'bidder_id')

In [80]:
final_test_df = final_test_df.merge(counts, how = 'left', on='bidder_id')

In [81]:
final_test_df = final_test_df.merge(bids,how='left', on ='bidder_id')

In [82]:
final_test_df.fillna(0,inplace=True)

In [83]:
final_test_df['country'].replace({0:1},inplace=True)

In [84]:
final_test_df['bids_per_auction'] = final_test_df['num_bids']/final_test_df['auction']
final_test_df['bids_per_url'] = final_test_df['num_bids']/final_test_df['url']
final_test_df['bids_per_ip'] = final_test_df['num_bids']/final_test_df['ip']
final_test_df['bids_per_country'] = final_test_df['num_bids']/final_test_df['country']
final_test_df['bids_per_device'] = final_test_df['num_bids']/final_test_df['device']

In [85]:
final_test_df = final_test_df.merge(country_bids,on='bidder_id',how='left')
final_test_df = final_test_df.merge(device_bids,on='bidder_id',how='left')
final_test_df = final_test_df.merge(ip_bids,on='bidder_id',how='left')
final_test_df = final_test_df.merge(url_bids,on='bidder_id',how='left')

In [86]:
final_test_df['check_first'] = final_test_df['bidder_id'].apply(func = check_first_bid)
final_test_df['check_last'] = final_test_df['bidder_id'].apply(func = check_last_bid)

In [87]:
testing_feature =['auction',
       'merchandise', 'device', 'time', 'country', 'ip', 'url', 'num_bids',
       'maxtime_num', 'mintime_num', 'maxdiff_num', 'mindiff_num',
       'meandiff_num', 'mediandiff_num', 'std_diff', 'median_start',
       'mean_start', 'std_start', 'median_end', 'mean_end', 'std_end',
       'mean_entropy', 'median_entropy', 'std_entropy',
       'median_ips_per_bidder_per_auction', 'mean_ips_per_bidder_per_auction',
       'std_ips_per_bidder_per_auction', 'ip_entropy',
       'median_country_per_bidder_per_auction',
       'mean_country_per_bidder_per_auction',
       'std_country_per_bidder_per_auction', 'country_entropy',
       'median_devices_per_bidder_per_auction',
       'mean_devices_per_bidder_per_auction',
       'std_devices_per_bidder_per_auction', 'device_entropy',
       'median_url_per_bidder_per_auction', 'mean_url_per_bidder_per_auction',
       'std_url_per_bidder_per_auction', 'url_entropy', 'bids_per_auction',
       'bids_per_url', 'bids_per_ip', 'bids_per_country', 'bids_per_device',
       'mean_country_per_auction', 'max_country_per_auction',
       'min_country_per_auction', 'std_country_per_auction',
       'mean_devices_per_auction', 'max_devices_per_auction',
       'min_devices_per_auction', 'std_devices_per_auction',
       'mean_ip_per_auction', 'max_ip_per_auction', 'min_ip_per_auction',
       'std_ip_per_auction', 'mean_url_per_auction', 'max_url_per_auction',
       'min_url_per_auction', 'std_url_per_auction', 'check_first',
       'check_last']

In [88]:
X_train = X
y_train = y

In [89]:
X_test = final_test_df[testing_feature]
X_test.fillna(0, inplace=True)

KeyError: "['mean_end', 'median_end', 'std_end', 'mean_start', 'median_start', 'std_start'] not in index"

In [ ]:
fmodel1= RandomForestClassifier(n_estimators=500, max_features=35,
                                   max_depth=8, random_state=20,
                                   criterion='entropy',)
fmodel2= GradientBoostingClassifier(n_estimators=200,
                                       random_state=20,
                                       max_depth=5,
                                       learning_rate=0.03,
                                       max_features=5,)
fmodel3 = xgb.XGBClassifier( objective= 'binary:logistic', nthread= 10,
                              eval_metric= 'auc', silent= 1, seed= 20,

                              max_depth= 6, gamma= 0, base_score= 0.50,
                              min_child_weight= 4, subsample= 0.5,
                              colsample_bytree= 1, eta= 0.01,)
fmodel4= CatBoostClassifier(random_state=20)


fmodel1=fmodel1.fit(X_train, y_train)
fmodel2=fmodel2.fit(X_train,y_train)
fmodel3=fmodel3.fit(X_train,y_train)
fmodel4=fmodel4.fit(X_train, y_train)

final_answer1= fmodel1.predict_proba(X_test)[:,1]
final_answer2= fmodel2.predict_proba(X_test)[:,1]
final_answer3= fmodel3.predict_proba(X_test)[:,1]
final_answer4= fmodel4.predict_proba(X_test)[:,1]


finalpredd= (final_answer1+final_answer2+final_answer3+final_answer4)/4
print(len(finalpredd))

In [ ]:
output_df= pd.DataFrame({
    'bidder_id': final_test_df.bidder_id,
    'Prediction':finalpredd
})
output_df

In [ ]:
output_df.to_csv('facebooksubmission.csv', index= False)